In [1]:
# EDIT:
base_dir = '/glade/work/kjmayer/research/catalyst/TransferLearning/runmean_analysis/artificial_bias/perfectmodel_TLtest/E3SM_analysis/'

In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import random
import pandas as pd
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
from tensorflow import keras
from tensorflow.keras import layers
import time

import sys
sys.path.append(base_dir+'functions/')
from utils import split_retrain
from exp_hp import get_hp

2025-01-27 15:44:07.395859: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-27 15:44:09.378701: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Num GPUs Available:  0


### Load standardized CESM2-LE

In [10]:
%%time
EXP_NAME = 'exp2'
EXP_NAME2 = 'exp2_retrain_increase'
hps = get_hp(EXP_NAME)
hps2 = get_hp(EXP_NAME2)

all_confcuttoffs = []

for i_trainmems in np.arange(0,9):
    print(i_trainmems)
    print('Training Members: '+str(hps2['train_mems'][i_trainmems]))
    print('Validation Member: '+str(hps2['val_mems'][0]))
    print('Testing Member: '+str(hps2['test_mems'][0]))
    print('Lead: '+str(hps2['LEAD']))
    print('Region: '+str(hps2['region'][0])+'N x '+str(hps2['region'][1])+'E')

    _, _, X1test, _, _, Y1test,_ = split_retrain(trainmems = hps2['train_mems'][i_trainmems],
                                                  valmem = hps2['val_mems'][0],
                                                  testmem = hps2['test_mems'][0],
                                                  months = [11,12,1,2], # months for X (Y+leadtime are accounted for in function)
                                                  lead = hps2['LEAD'])

    X1test = X1test.reshape((X1test.shape[0],X1test.shape[1]*X1test.shape[2]))
    
    
    #----- SET UP & TRAIN NN -----
    GLOBAL_SEED = hps['GLOBAL_SEED']
    np.random.seed(GLOBAL_SEED)
    random.seed(GLOBAL_SEED)
    tf.random.set_seed(GLOBAL_SEED)

    # acc_all = []
    for SEED in np.arange(0,20):
        print(SEED)
        
        annfi_name = 'ann2_60Eshift_'+EXP_NAME2+'.'+str(i_trainmems)+'_ann1-'+EXP_NAME+'_seed'+str(SEED)+'.h5'
        ann = tf.keras.models.load_model(base_dir+'train/saved_models/'+annfi_name)

        pred = ann.predict(X1test)
        acc = []
        predconf = np.max(pred,axis=-1)
        predval  = np.argmax(pred,axis=-1)

        # for q in range(100):
        q = 80
        all_confcuttoffs.append(np.percentile(predconf,q=q))
        #     iconf = np.where(predconf >= np.percentile(predconf,q=q))
        #     icorr = np.where(predval[iconf] == Y1test[iconf])

        #     acc.append((len(icorr[0])/len(iconf[0]))*100)
        # plt.plot(acc)
        # plt.ylim(50,100)

        # acc_all.append(acc)
    # plt.show()
    print(all_confcuttoffs)
    # np.save(base_dir+'test/confvsacc/confvsacc_ann2_60Eshift_'+EXP_NAME2+'.'+str(i_trainmems)+'_ann1-'+EXP_NAME+'_20seeds.npy', acc_all, allow_pickle=True)

0
Training Members: ['0201']
Validation Member: 0291
Testing Member: 0301
Lead: 14
Region: 30-60N x 170-240E
loading data & saving
made it!
0
228/228 [==============================] - 0s 1ms/step
1
228/228 [==============================] - 0s 2ms/step
2
228/228 [==============================] - 0s 1ms/step
3
228/228 [==============================] - 0s 1ms/step
4
228/228 [==============================] - 0s 1ms/step
5
228/228 [==============================] - 0s 1ms/step
6
228/228 [==============================] - 0s 1ms/step
7
228/228 [==============================] - 0s 1ms/step
8
228/228 [==============================] - 0s 2ms/step
9
228/228 [==============================] - 0s 2ms/step
10
228/228 [==============================] - 0s 1ms/step
11
228/228 [==============================] - 0s 1ms/step
12
228/228 [==============================] - 0s 1ms/step
13
228/228 [==============================] - 0s 2ms/step
14
228/228 [==============================] - 0s 1ms/step


In [12]:
np.mean(all_confcuttoffs[-20:])

0.5992882078886032